<a href="https://colab.research.google.com/github/multi-proj/API/blob/master/Soccer_Team/Team_Sheet_20190514.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from selenium import webdriver

In [0]:
!pip install selenium
!apt install chromium-chromedriver

In [5]:
!pip install -U -q Pydrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

     |████████████████████████████████| 993kB 4.9MB/s 


In [0]:
url = 'https://drive.google.com/open?id=1dwaiV1_fIVBGHGeVKnRGXMgw-POve3I3'
id = url.split('=')[1]

In [0]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
wd.get("https://www.premierleague.com/players?se=79&cl=21")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


---

* **선수명 불러오기**

In [0]:
player_list = wd.find_elements_by_class_name('playerName')

In [0]:
player_name = []

for i in range(0, len(player_list)):
    name = player_list[i].text
    player_name.append(name)
    
player_name

---

* **포지션 불러오기**

In [0]:
player_position_name = wd.find_elements_by_class_name('hide-s')

In [0]:
player_position = []

for i in range(2, len(player_position_name), 2):
    name = player_position_name[i].text
    player_position.append(name)

player_position

---

* **선수이름 + 포지션**

In [0]:
import pandas as pd

data = {'Name': player_name, 'Position': player_position}

team_sheet = pd.DataFrame(data)
team_sheet.head()

team_sheet = team_sheet.drop(49)

---

* **선수 링크 긁어오기**

In [0]:
test = wd.find_elements_by_tag_name('a')
test

In [0]:
link = []
for i in range(0, len(test)):
    a = test[i].get_attribute('href')
    link.append(a)
    
link

In [0]:
link = link[109:158]

In [0]:
link_number = []

for i in range(0, len(link)):
               a = int(link[i].split('/')[4])
               link_number.append(a)

            
# 선수 테이블과 합치기
team_sheet['id'] = link_number

---

## **지난 경기 기록 긁어오기**

In [0]:
!pip install selenium
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
match_wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
match_wd.get("https://www.premierleague.com/clubs/21/Tottenham-Hotspur/results?co=1&se=79")

In [0]:
# match_tag = match_wd.find_elements_by_tag_name('li')

match_num_01 = []

for i in range(1, 39):
    a = i * 3
    number = '#mainContent > div:nth-child(6) > div > div.wrapper.col-12.active > div.col-12 > section > div:nth-child(%d) > ul > li > div' % a
    b = match_wd.find_element_by_css_selector(number)
    
    match_num_01.append(b)

In [0]:
match_num_02 = []

for i in range(0, len(match_num_01)):
    a = match_num_01[i].get_attribute('data-matchid')
    match_num_02.append(a)
    
match_num_02 = list(map(int,match_num_02))
match_num_02

In [48]:
match_num_03 = []
for i in match_num_02:
    a = 'https://www.premierleague.com/match/%d' % i
    match_num_03.append(a)
    
match_num_03

['https://www.premierleague.com/match/22720',
 'https://www.premierleague.com/match/22650',
 'https://www.premierleague.com/match/22711',
 'https://www.premierleague.com/match/22700',
 'https://www.premierleague.com/match/22684',
 'https://www.premierleague.com/match/22680',
 'https://www.premierleague.com/match/22669',
 'https://www.premierleague.com/match/22654',
 'https://www.premierleague.com/match/22633',
 'https://www.premierleague.com/match/22630',
 'https://www.premierleague.com/match/22616',
 'https://www.premierleague.com/match/22610',
 'https://www.premierleague.com/match/22598',
 'https://www.premierleague.com/match/22591',
 'https://www.premierleague.com/match/22578',
 'https://www.premierleague.com/match/22569',
 'https://www.premierleague.com/match/22549',
 'https://www.premierleague.com/match/22560',
 'https://www.premierleague.com/match/22539',
 'https://www.premierleague.com/match/22524',
 'https://www.premierleague.com/match/22518',
 'https://www.premierleague.com/ma

---

## **경기별 선수 명단 긁어오기**

In [0]:
# https://www.premierleague.com/match/22720
from bs4 import BeautifulSoup
import urllib.request
import urllib.parse

# web_url에 원하는 웹의 URL을 넣어주시면 됩니다.

players = []

for i in match_num_03:

    with urllib.request.urlopen(i) as response:
        html = response.read()
        soup = BeautifulSoup(html, 'html.parser')
        players.append(soup.find_all('div', 'matchLineupTeamContainer'))


<div class="matchLineupTeamContainer" data-index-class="home" data-team-id="21"><ul class="startingLineUpContainer squadList home"><li class="player" data-age="535939200000" data-position="0"><a href="/players/4664/Hugo-Lloris/overview"><div class="number">1</div><img alt="Hugo Lloris" class="img" data-player="p37915" data-script="pl_player-image" data-size="110x140" data-widget="player-image" src="//platform-static-files.s3.amazonaws.com/premierleague/photos/players/110x140/Photo-Missing.png"/><div class="info"><div class="name"> Hugo Lloris <div class="playerLineupEventContainer-4,664"></div></div><span class="position"><div class="cpt">C</div><span>Goalkeeper</span></span></div></a></li><li class="player" data-age="758592000000" data-position="1"><a href="/players/4112/Eric-Dier/overview"><div class="number">15</div><img alt="Eric Dier" class="img" data-player="p93264" data-script="pl_player-image" data-size="110x140" data-widget="player-image" src="//platform-static-files.s3.amazon

In [73]:
import pandas as pd
import numpy as np

players_name = []

df = pd.DataFrame(players_name, columns = ['Name', 'Goal'])
df

# df.loc[j, 'Name'] = [players_name]
# df.loc[j, 'Goal'] = [players_goal]

,Name,Goal


In [81]:
player_name = []

for i in range(0, len(players)) :
    if (players[i].get_attribute_list('data-team-id')[0] == '21') :
        a = players[i].find_all("div", class_="name")
       # print(a)
        for j in range(0, len(a)) :
            player_name.append(a[j].get_text())

player_name

AttributeError: ignored

In [78]:
players_name = []
players_goal = []

for i in range(0, len(players)) :
    if (players[i].get_attribute_list('data-team-id')[0] == '21'):
        a = players[i].find_all("div", class_="name")
    elif (players[i].get_attribute_list('data-team-id')[1] == '21'):
    
        for j in range(0, len(a)) :
            players_name = a[j].get_text()
            players_goal = int(len(a[j].find_all("span", class_="icn ball-small")))
            
            df.loc[j, 'Name'] = [players_name]
            df.loc[j, 'Goal'] = [players_goal]

df

AttributeError: ignored

* **골 뽑아오기**

In [45]:
players_name = []

for i in range(0, len(players)) :
    if (players[i].get_attribute_list('data-team-id')[0] == '21') :
        a = players[i].find_all("div", class_="name")
        for j in range(0, len(a)):
            print(len(a[j].find_all("span", class_="icn ball-small")))

0
0
0
0
0
0
2
0
0
0
2
0
0
0
0
0
0
0


In [20]:
lstrip_players_name = []

for i in range(0, len(players_name)):
    a = player_name[i].lstrip()
    lstrip_players_name.append(a)
    
lstrip_player_name

['Hugo Lloris ',
 'Eric Dier ',
 'Kyle Walker-Peters ',
 'Danny Rose ',
 'Toby Alderweireld ',
 'Victor Wanyama ',
 "Erik Lamela 78'",
 "Lucas Moura 74'",
 "Moussa Sissoko 84'",
 'Christian Eriksen ',
 'Harry Kane ',
 'Michel Vorm ',
 'Ben Davies ',
 'Juan Foyth ',
 "Davinson Sánchez 78'",
 "Dele Alli 74'",
 "Son Heung-Min 84'",
 'Fernando Llorente ']

* **정규식**
    * 출처1: http://melonicedlatte.com/python/2017/02/23/152309.html
    * 출처2: https://niceman.tistory.com/156

In [22]:
re_players_name = []
fin_players_name = []

import re

for i in range(0, len(lstrip_players_name)):
    text = lstrip_players_name[i]
    parse = re.sub('[' '][0-9][0-9][-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', text)
    re_players_name.append(parse)

for j in range(0, len(re_players_name)):
    a = re_players_name[j].rstrip()
    fin_players_name.append(a)
        
fin_players_name

#     for j in range(0, len(re_player_name)):
#         a = re_player_name[j].rstrip()
#         fin_player_name.append(a)

['Hugo Lloris',
 'Eric Dier',
 'Kyle Walker-Peters',
 'Danny Rose',
 'Toby Alderweireld',
 'Victor Wanyama',
 'Erik Lamela',
 'Lucas Moura',
 'Moussa Sissoko',
 'Christian Eriksen',
 'Harry Kane',
 'Michel Vorm',
 'Ben Davies',
 'Juan Foyth',
 'Davinson Sánchez',
 'Dele Alli',
 'Son Heung-Min',
 'Fernando Llorente']